In [1]:
import gym
import gym_thing
import math
import gym_thing.kinematics.kinematics as K
from pyquaternion import Quaternion
import numpy as np
import json

In [ ]:
def forward_base(theta):
    T_odom2rb = np.array([
        [math.cos(theta[8]), -math.sin(theta[8]), 0, theta[6]],
        [math.sin(theta[8]), math.cos(theta[8]), 0, theta[7]],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    T_rb2arch = np.array([
        [1, 0, 0, 0.27 ],
        [0, 1, 0, 0.01 ],
        [0, 0, 1, 0.653],
        [0, 0, 0, 1]
    ])
    return T_odom2rb.dot(T_rb2arch)

def get_pos_rot(env, name):
    sim_pos = env.sim.data.get_body_xpos(name)
    sim_quat = env.sim.data.get_body_xquat(name)
    q = Quaternion(sim_quat)
    print(sim_pos)
    print(q.rotation_matrix)
    return sim_pos, q.rotation_matrix

In [ ]:
env = gym.make('thing-v0')
env.reset()
theta = env.get_obs()[:8].tolist() + [0]
env.render()

In [ ]:
print(K.forward_kinematics_odom2tcp(theta))

In [ ]:
pos, rot = get_pos_rot(env, 'thing_tool')

In [16]:
def reprocess_json(input_json, output_file):
    result = []
    with open(input_json, 'r') as fid:
        j = json.load(fid)
    for element in j:
        entry = {}
        config = element['config']
        entry['ur10_states'] = config[:6]
        entry['base_states'] = config[6:]
        T = np.reshape(element['matrix'], (3, 4))
        entry['tcp_states'] = [T[0, 3], T[1, 3], T[2, 3]]
        entry['tcp_z_axis'] = [T[0, 1], T[1, 1], T[2, 1]]
        result.append(entry)
    output = {}
    output['number'] = len(result)
    output['data'] = result
    
    with open(output_file, 'w') as fid:
        json.dump(output, fid, indent=4)

In [17]:
reprocess_json('forward_kinematics_samples.json', 'forward_kinematics_calibration.json')